In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Fibromyalgia"
cohort = "GSE67311"

# Input paths
in_trait_dir = "../../input/GEO/Fibromyalgia"
in_cohort_dir = "../../input/GEO/Fibromyalgia/GSE67311"

# Output paths
out_data_file = "../../output/preprocess/Fibromyalgia/GSE67311.csv"
out_gene_data_file = "../../output/preprocess/Fibromyalgia/gene_data/GSE67311.csv"
out_clinical_data_file = "../../output/preprocess/Fibromyalgia/clinical_data/GSE67311.csv"
json_path = "../../output/preprocess/Fibromyalgia/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# From the background information, we can see that Affymetrix Human Gene arrays were used
# and gene expression analysis was performed, so gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Fibromyalgia)
# From sample characteristics, we see 'diagnosis' in key 0 
# with values 'healthy control' and 'fibromyalgia'
trait_row = 0

# For age - There is no age information in the sample characteristics
age_row = None

# For gender - There is no gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

# Function to convert trait values
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Convert to binary (0 for control, 1 for fibromyalgia)
    if value == 'fibromyalgia':
        return 1
    elif value == 'healthy control':
        return 0
    return None

# Age conversion function (not used as age is not available)
def convert_age(value):
    return None

# Gender conversion function (not used as gender is not available)
def convert_gender(value):
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial filtering results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    # Create the clinical data DataFrame from the Sample Characteristics Dictionary provided earlier
    sample_characteristics_dict = {
        0: ['diagnosis: healthy control', 'diagnosis: fibromyalgia'], 
        1: ['tissue: peripheral blood'], 
        2: ['fiqr score: 8.5', 'fiqr score: -2.0', 'fiqr score: 9.8', 'fiqr score: 0.5', 'fiqr score: -1.0', 'fiqr score: -0.5', 'fiqr score: 2.2', 'fiqr score: 15.3', 'fiqr score: 4.0', 'fiqr score: 29.3', 'fiqr score: 27.2', 'fiqr score: 5.0', 'fiqr score: 1.0', 'fiqr score: 2.5', 'fiqr score: 3.0', 'fiqr score: -1.5', 'fiqr score: 1.3', 'fiqr score: 21.7', 'fiqr score: -1.2', 'fiqr score: 4.3', 'fiqr score: 6.5', 'fiqr score: 2.0', 'fiqr score: 11.7', 'fiqr score: 15.0', 'fiqr score: 6.0', 'fiqr score: 14.2', 'fiqr score: -0.2', 'fiqr score: 12.8', 'fiqr score: 15.7', 'fiqr score: 0.0'], 
        3: ['bmi: 36', 'bmi: 34', 'bmi: 33', 'bmi: 22', 'bmi: 24', 'bmi: 28', 'bmi: 23', 'bmi: 48', 'bmi: 25', 'bmi: 46', 'bmi: 32', 'bmi: 31', 'bmi: 21', 'bmi: 27', 'bmi: 39', 'bmi: 52', 'bmi: 37', 'bmi: 0', 'bmi: 38', 'bmi: 26', 'bmi: 42', 'bmi: 20', 'bmi: 30', 'bmi: 43', 'bmi: 35', 'bmi: 44', 'bmi: 29', 'bmi: 45', 'bmi: 40', 'bmi: 47'], 
        4: ['migraine: No', 'migraine: Yes', 'migraine: -'], 
        5: ['irritable bowel syndrome: No', 'irritable bowel syndrome: Yes', 'irritable bowel syndrome: -'], 
        6: ['major depression: No', 'major depression: -', 'major depression: Yes'], 
        7: ['bipolar disorder: No', 'bipolar disorder: -', 'bipolar disorder: Yes'], 
        8: ['chronic fatigue syndrome: No', np.nan, 'chronic fatigue syndrome: -', 'chronic fatigue syndrome: Yes']
    }
    
    clinical_data = pd.DataFrame({k: pd.Series(v) for k, v in sample_characteristics_dict.items()})
    
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    print("Preview of clinical features:")
    print(preview_df(clinical_features))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
# Let's examine whether the dataset contains the necessary information
print("Examination of GSE67311 dataset for Fibromyalgia study")

# First, let's check if the files exist
clinical_data_file = os.path.join(in_cohort_dir, "clinical_data.csv")
meta_data_file = os.path.join(in_cohort_dir, "meta_data.json")

# Initialize flags for data availability
is_gene_available = False
is_trait_available = False

# Initialize variables
clinical_data = None
meta_data = {}
trait_row = None
age_row = None
gender_row = None

# Try to load clinical data
if os.path.exists(clinical_data_file):
    clinical_data = pd.read_csv(clinical_data_file)
    print("Clinical data shape:", clinical_data.shape)
    print("Clinical data columns:", clinical_data.columns.tolist())
    print("Sample of clinical data:\n", clinical_data.head())
else:
    print(f"Clinical data file not found at: {clinical_data_file}")
    print("Checking for alternative files in the directory...")
    
    # Check if there are any CSV files in the directory
    csv_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('.csv')]
    if csv_files:
        print(f"Found CSV files: {csv_files}")
        # Try the first CSV file
        alternative_file = os.path.join(in_cohort_dir, csv_files[0])
        try:
            clinical_data = pd.read_csv(alternative_file)
            print(f"Loaded alternative clinical data from: {alternative_file}")
            print("Clinical data shape:", clinical_data.shape)
            print("Sample of clinical data:\n", clinical_data.head())
        except Exception as e:
            print(f"Error loading alternative file: {e}")
    else:
        print("No CSV files found in the directory.")

# Try to load meta data
if os.path.exists(meta_data_file):
    with open(meta_data_file, 'r') as f:
        meta_data = json.load(f)
    print("Meta data keys:", list(meta_data.keys()))
    
    if 'title' in meta_data:
        print("Dataset title:", meta_data.get('title'))
    
    if 'background' in meta_data:
        print("Background information:", meta_data.get('background'))
    
    # Check for gene expression data availability based on meta_data
    if any(keyword in str(meta_data).lower() for keyword in 
           ['gene expression', 'mrna', 'transcriptome', 'gene profile']):
        is_gene_available = True
    
    if 'sample_characteristics' in meta_data:
        sample_chars = meta_data.get('sample_characteristics', {})
        print("Sample characteristics keys:", list(sample_chars.keys()))
        
        # Print the unique values for each key in sample characteristics
        for key, values in sample_chars.items():
            unique_values = set(values)
            print(f"Key {key} unique values:", unique_values)
            
            # Check for trait, age, and gender data
            if any('fibromyalgia' in str(v).lower() or 'fm' in str(v).lower() or trait.lower() in str(v).lower() 
                  for v in unique_values):
                trait_row = int(key)
                is_trait_available = True
            
            if any('age' in str(v).lower() for v in unique_values):
                age_row = int(key)
            
            if any('gender' in str(v).lower() or 'sex' in str(v).lower() or 
                  'female' in str(v).lower() or 'male' in str(v).lower() for v in unique_values):
                gender_row = int(key)
else:
    print(f"Meta data file not found at: {meta_data_file}")
    print("Checking for alternative JSON files in the directory...")
    
    # Check if there are any JSON files in the directory
    json_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('.json')]
    if json_files:
        print(f"Found JSON files: {json_files}")
        # Try the first JSON file
        alternative_file = os.path.join(in_cohort_dir, json_files[0])
        try:
            with open(alternative_file, 'r') as f:
                meta_data = json.load(f)
            print(f"Loaded alternative meta data from: {alternative_file}")
        except Exception as e:
            print(f"Error loading alternative JSON file: {e}")
    else:
        print("No JSON files found in the directory.")

# Check for data in any other files in the directory
if not is_gene_available:
    # Look for files that might contain gene expression data
    gene_data_indicators = ['gene', 'expression', 'probe', 'mrna', 'matrix', 'series']
    all_files = os.listdir(in_cohort_dir)
    potential_gene_files = [f for f in all_files if any(indicator in f.lower() for indicator in gene_data_indicators)]
    
    if potential_gene_files:
        print(f"Found potential gene expression files: {potential_gene_files}")
        is_gene_available = True
    else:
        print("No files indicating gene expression data found.")

# Output the identified rows
print(f"Identified trait_row: {trait_row}")
print(f"Identified age_row: {age_row}")
print(f"Identified gender_row: {gender_row}")
print(f"Is gene expression data available: {is_gene_available}")
print(f"Is trait data available: {is_trait_available}")

# Define conversion functions regardless of data availability
def convert_trait(value):
    if value is None:
        return None
    
    value = str(value).lower()
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for control, 1 for fibromyalgia)
    if 'fibromyalgia' in value or 'fm' in value or 'patient' in value:
        return 1
    elif 'control' in value or 'healthy' in value:
        return 0
    else:
        return None

def convert_age(value):
    if value is None:
        return None
    
    value = str(value).lower()
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric age using regex
    import re
    match = re.search(r'(\d+(\.\d+)?)', value)
    if match:
        return float(match.group(1))
    else:
        return None

def convert_gender(value):
    if value is None:
        return None
    
    value = str(value).lower()
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for female, 1 for male)
    if 'female' in value or 'f' == value.strip():
        return 0
    elif 'male' in value or 'm' == value.strip():
        return 1
    else:
        return None

# Use validate_and_save_cohort_info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if data is available
if trait_row is not None and clinical_data is not None:
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row


### Step 4: Gene Data Extraction

In [ ]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True


### Step 5: Gene Identifier Review

In [ ]:
# Analyzing the gene identifiers from the previous step

# These appear to be microarray probe IDs (likely Affymetrix Human Gene 1.0 ST Array)
# They are 7-digit numeric IDs (7892501, 7892502, etc.) which are typical for
# probesets in microarray platforms, not standard human gene symbols
# Human gene symbols would be alphanumeric (like BRCA1, TP53, etc.)

# Since these are probe IDs and not human gene symbols, they will need to be mapped
# to standard gene symbols for proper biological interpretation

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns that contain gene identifiers and gene symbols
# From the gene annotation preview, I can see:
# - 'ID' column has identifiers like '7896736' which match the format in gene expression data
# - 'gene_assignment' column contains gene symbols and annotations

# 2. Create a gene mapping dataframe
# The 'gene_assignment' column contains complex text with gene symbols
# Using get_gene_mapping to extract IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Print the mapping dataframe to verify
print("Gene mapping dataframe preview:")
print(preview_df(mapping_df))

# 3. Convert probe-level measurements to gene expression data
# Using the library function to apply gene mapping
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the dimensions and preview of the gene expression data after mapping
print(f"\nGene expression data after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First few rows of the mapped gene expression data:")
print(preview_df(gene_data))

# Normalize gene symbols
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nAfter normalizing gene symbols: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Extract the gene expression data from the matrix file 
gene_expression_data = get_genetic_data(matrix_file)

# Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Apply gene mapping and normalize gene symbols
gene_data = apply_gene_mapping(gene_expression_data, mapping_df)
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# 1. Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Process clinical data
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Determine trait row (from previous step)
trait_row = 0  # 'diagnosis: healthy control' or 'diagnosis: fibromyalgia'

# Define conversion function for trait
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Convert to binary (0 for control, 1 for fibromyalgia)
    if value == 'fibromyalgia':
        return 1
    elif value == 'healthy control':
        return 0
    return None

# Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical features preview:")
print(preview_df(clinical_features))

# 2. Link clinical and genetic data
if not clinical_features.empty:
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    # 3. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine if trait and demographic features are biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Data shape after removing biased features: {linked_data.shape}")
    
    # 5. Validate and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note="Dataset contains gene expression data from peripheral blood of Fibromyalgia patients and healthy controls."
    )
    
    # 6. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable for associational studies.")
else:
    # No clinical data available
    print("Clinical data is empty. Dataset not usable for association studies.")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=pd.DataFrame(index=normalized_gene_data.columns),
        note="Dataset contains gene expression data but lacks usable clinical metadata for Fibromyalgia studies."
    )